In [ ]:
!pip install tensorflow keras numpy pandas opencv-python kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset

In [ ]:
!unzip /content/brain-tumor-mri-dataset.zip -d dataset

In [ ]:
!pip install torchvision scikit-learn matplotlib

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
  train_dir = "/content/dataset/Training"
test_dir = "/content/dataset/Testing"
train_dataset = ImageFolder(root=train_dir, transform=train_transform)
test_dataset = ImageFolder(root=test_dir, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm
from google.colab import drive


In [ ]:
# Hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 10


In [ ]:

# Define Dataset Paths
train_path = "/content/dataset/training"
test_path = "/content/dataset/testing"


In [ ]:
# Define Data Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor(),         # Convert to Tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])


In [ ]:
# Helper Function to Define Models
def create_model(model_name):
    if model_name == "resnet50":
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))
    elif model_name == "efficientnet_b0":
        model = EfficientNet.from_pretrained("efficientnet-b0")
        model._fc = nn.Linear(model._fc.in_features, len(train_dataset.classes))

    elif model_name == "densenet":
        model = models.densenet121(pretrained=True)
        model.classifier = nn.Linear(model.classifier.in_features, len(train_dataset.classes))

    return model.to(device)


In [ ]:
# Models to Train
models_to_train = ["resnet50", "efficientnet_b0", "densenet"]



In [ ]:
# Train Function
def train_model(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward Pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward Pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return running_loss / len(train_loader), accuracy



In [ ]:
# Test Function
def test_model(model, test_loader):
    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_predictions

In [ ]:
!pip install efficientnet_pytorch


In [ ]:
from efficientnet_pytorch import EfficientNet


In [ ]:
# Train and Evaluate All Models
trained_models = {}
for model_name in models_to_train:
    print(f"Training {model_name}...")
    model = create_model(model_name)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    epoch_accuracies = []
    for epoch in range(num_epochs):
        train_loss, train_accuracy = train_model(model, train_loader, criterion, optimizer)
        print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")
        epoch_accuracies.append(train_accuracy)

    # Evaluate Model on Test Set
    labels, predictions = test_model(model, test_loader)
    acc = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='macro')
    f1 = f1_score(labels, predictions, average='macro')
    cm = confusion_matrix(labels, predictions)

    print(f"Test Accuracy for {model_name}: {acc:.2f}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"Precision: {precision:.2f}, F1 Score: {f1:.2f}")

    # Save Model
    model_save_path = f"/content/drive/MyDrive/{model_name}_brain_tumor.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")

    # Plot Epoch vs Accuracy
    plt.plot(range(1, num_epochs + 1), epoch_accuracies, label=model_name)

    # Store Trained Model
    trained_models[model_name] = model



Training resnet50...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 58.5MB/s]
100%|██████████| 179/179 [57:19<00:00, 19.21s/it]


Epoch [1/10] - Loss: 0.5282, Accuracy: 79.52%


  4%|▍         | 7/179 [02:18<56:24, 19.67s/it]

In [ ]:
def ensemble_predictions(models, test_loader):
    all_ensemble_preds = []
    all_labels = []

    for images, labels in tqdm(test_loader):
        images = images.to(device)

        # Collect Predictions from All Models
        predictions = []
        for model_name, model in models.items():
            model.eval()
            with torch.no_grad():
                outputs = model(images)
                predictions.append(torch.softmax(outputs, dim=1).cpu().numpy())

        # Perform Mean Fusion
        fused_preds = np.mean(predictions, axis=0)
        fused_classes = np.argmax(fused_preds, axis=1)

        all_ensemble_preds.extend(fused_classes)
        all_labels.extend(labels.numpy())

    return all_labels, all_ensemble_preds

ensemble_labels, ensemble_predictions = ensemble_predictions(trained_models, test_loader)

In [ ]:
# Evaluate Ensemble Model
ensemble_acc = accuracy_score(ensemble_labels, ensemble_predictions)
ensemble_precision = precision_score(ensemble_labels, ensemble_predictions, average='macro')
ensemble_f1 = f1_score(ensemble_labels, ensemble_predictions, average='macro')
ensemble_cm = confusion_matrix(ensemble_labels, ensemble_predictions)

print(f"\nEnsemble Model Results:")
print(f"Accuracy: {ensemble_acc:.2f}")
print(f"Precision: {ensemble_precision:.2f}, F1 Score: {ensemble_f1:.2f}")
print(f"Confusion Matrix:\n{ensemble_cm}")



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd

# Assuming `labels` and `predictions` are already defined
# Replace the following with your actual data if needed
# labels = [...]         # True labels
# predictions = [...]    # Model predictions

# Calculate metrics
accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='macro')  # Adjust 'average' as needed
recall = recall_score(labels, predictions, average='macro')
f1 = f1_score(labels, predictions, average='macro')
cm = confusion_matrix(labels, predictions)

# Create and display a table
metrics = {
    "Metric": ["Accuracy", "Precision", "Recall", "F1 Score"],
    "Value": [accuracy, precision, recall, f1]
}

metrics_df = pd.DataFrame(metrics)
print(metrics_df)

# (Optional) Print the confusion matrix separately
print("\nConfusion Matrix:")
print(cm)


In [ ]:
import seaborn as sns

# Plot Confusion Matrix
plt.figure(figsize=(10,8))
sns.heatmap(ensemble_cm, annot=True, fmt='d', cmap='Blues', xticklabels=train_dataset.classes, yticklabels=train_dataset.classes)
plt.title('Confusion Matrix for Ensemble Model')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()
